In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader

# Load and preprocess the dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Prepare train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["text"].fillna(""), train_data["target"], test_size=0.2, random_state=42
)

# Dataset class
class DisasterTweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = str(self.labels.iloc[idx]) if self.labels is not None else None
        inputs = self.tokenizer(
            text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        if label is not None:
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(
                    label, max_length=2, padding="max_length", truncation=True, return_tensors="pt"
                )
            inputs["labels"] = labels["input_ids"].squeeze()
        return {key: val.squeeze() for key, val in inputs.items()}


In [12]:
# T5-Small Model Implementation

def train_t5_small():
    # Load model and tokenizer
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Prepare datasets
    train_dataset = DisasterTweetDataset(train_texts, train_labels, tokenizer)
    val_dataset = DisasterTweetDataset(val_texts, val_labels, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    # Set up optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Training loop
    for epoch in range(3):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            labels = batch["labels"].to(device)
            outputs = model.generate(**inputs)

            # Decode predictions and labels
            batch_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            batch_true_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Filter out empty or invalid predictions/labels
            batch_predictions = [pred.strip() for pred in batch_predictions if pred.strip().isdigit()]
            batch_true_labels = [true.strip() for true in batch_true_labels if true.strip().isdigit()]

            predictions.extend(batch_predictions)
            true_labels.extend(batch_true_labels)

    # Convert to integers
    predictions = list(map(int, predictions))
    true_labels = list(map(int, true_labels))

    # Compute F1 score
    if len(predictions) > 0 and len(true_labels) > 0:  # Ensure there are valid entries
        f1 = f1_score(true_labels, predictions)
        print(f"Epoch {epoch + 1}: F1 Score = {f1}")
    else:
        print("No valid predictions or labels were generated for evaluation.")


train_t5_small()


ValueError: Found input variables with inconsistent numbers of samples: [649, 663]

In [ ]:
# T5-Base Model Implementation

def train_t5_base():
    model_name = "t5-base"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    # The rest of the code is the same as T5-small implementation

train_t5_base()


In [ ]:
# T5-Large Model Implementation

def train_t5_large():
    model_name = "t5-large"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    # The rest of the code is the same as T5-small implementation

Train_t5_large()


In [ ]:
test_dataset = DisasterTweetDataset(test_data["text"].fillna(""), None, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        outputs = model.generate(**inputs)
        predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

submission = pd.DataFrame({"id": test_data["id"], "target": predictions})
submission["target"] = submission["target"].astype(int)
submission.to_csv("submission.csv_T5_small", index=False)
